This Notebook creates the PNG surface figures for the leave one out tract group analysis of the connectivity blueprints <br />
This corresponds to figures 6-9 and generates the figures shown in parts A and B. <br />
We show an example of how part C of the figures was also generated though those were visualized using connectome workbench
<br />
<br />
Please ensure all packages in the requirements.txt are installed before running<br/>

The implementation of this notebook is for the left hemisphere. <br />To run on the right comment out the left, and comment in the right

In [9]:
import numpy as np
import nibabel as nib
import scipy.io as sio
from scipy import stats
import pandas as pd
import h5py
import nilearn
from nilearn import plotting
import seaborn as sn
from math import pi
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, HTML
import mayavi
import matplotlib
import scipy
import warnings
from vtkmodules import *
from mayavi import mlab


%matplotlib inline
!jupyter nbextension install --py mayavi --user 

ModuleNotFoundError: No module named 'seaborn'

In [10]:
from mayavi import mlab

In [ ]:
## Load in the connectivity Blueprints
pig=h5py.File('./blueprints/L_pig27.mat','r')
variables=pig.items()
for var in variables:
    pig_name=var[0]
    pig_data=var[1]
    print(pig_name)
    print
    pig_data=np.array(pig_data).transpose()
    pig_data=pig_data
hum=h5py.File('./blueprints/L_hum27.mat','r')
variables=hum.items()
for var in variables:
    hum_name=var[0]
    print(hum_name)
    hum_data=var[1]
    hum_data=np.array(hum_data).transpose()



# ##### comment in to run on right hemisphere

# pig=h5py.File('./blueprints//R_pig27.mat','r')
# variables=pig.items()
# for var in variables:
#     pig_name=var[0]
#     pig_data=var[1]
#     print(pig_name)
#     print
#     pig_data=np.array(pig_data).transpose()
#     pig_data=pig_data
# hum=h5py.File('./blueprints//R_hum27.mat','r')
# variables=hum.items()
# for var in variables:
#     hum_name=var[0]
#     print(hum_name)
#     hum_data=var[1]
#     hum_data=np.array(hum_data).transpose()

In [ ]:
##### load in text file containing names of shared tracts between pig and human 
with open('./blueprints//structureList.txt','r') as structs:
    structs=structs.read()
    structs=structs.split('\n')
    ALL_dict={}
for i in range(len(structs)-1):
        ALL_dict[structs[i]]=hum_data[:,i]

In [ ]:
#### function to remove tracts from BP
def remove_tracts(BP,quitar):
    BP_new=dict(BP)
    orig=BP_new.keys()
    for i in range(len(quitar)):
        val=quitar[i]
        if val in orig:
            del BP_new[val]
    return BP_new

In [ ]:
## make pig and human BP's dictionaries
with open('./blueprints//structureList.txt','r') as structs:
    structs=structs.read()
    structs=structs.split('\n')
pig_27={}
hum_27={}
for i in range(len(structs)-1):
    pig_27[structs[i]]=pig_data[:,i]
    hum_27[structs[i]]=hum_data[:,i]
    

In [ ]:
### Function to calculate the KL divergence as done in the Mars blueprint paper
def calc_kl(A,B):
    if type(A) == dict:
        A=list(A.values())
        A=np.array(A).transpose()
    else:
            pass
    if type(B) == dict:
        B=list(B.values())
        B=np.array(B).transpose()
    else:
        pass
    
    p=A.shape
    p=p[0]

    ## function takes two numpy arrays that are the blueprints not normalized
    def normalize(BP,p):
        BP[(np.isnan(BP))]=0
        row_sums=BP.sum(axis=1)
        BP=BP/row_sums[:,np.newaxis]
        return BP
    
    A=normalize(A,p)
    B=normalize(B,p)
    Amask= A!=0
    A_invmask=Amask!=1
    Bmask= B !=0
    B_invmask=Bmask!=1
 
    ##### pyhton implementation of Saad Jaabdi's matlab code fo KL divergence
    KL=np.dot(np.multiply(A,(np.log2((A+A_invmask)))),Bmask.transpose()) \
    - np.dot(A,(Bmask*np.log2(B+B_invmask)).transpose()) \
    + np.dot(Amask,(B*np.log2(B+B_invmask)).transpose())\
    - np.dot(Amask*np.log2(A+A_invmask),B.transpose())
    return KL
    

In [ ]:
#### function defining the plotting of vectors on surfaces
#### requires mayavi!!! 
def oh_mayavi(surf,stat,cmap,vmi,vma,*args):
    ##### parse the gifti 
    anat=nib.load(surf)
    coords=anat.darrays[0].data
    x=coords[:,0]
    y=coords[:,1]
    z=coords[:,2]
    triangles=anat.darrays[1].data
    ##### if subcortical mask provided use it
    if len(args) >0:
#         print('masking out subcortex')
        sub_cort=nilearn.surface.load_surf_data(args[0])
        stat[sub_cort]=float('NaN')
    else:
        pass
    
    
    ### start mayavi 
    mlab.init_notebook('png',1500,1500)
    maya=mlab.triangular_mesh(x,y,z,triangles,scalars=stat,colormap=cmap,vmin=vmi,vmax=vma)
    
    mlab.view(azimuth=0, elevation=-90)
    f = mlab.gcf()
    cam = f.scene.camera
    cam.zoom(1.)
#     cb=mlab.colorbar(orientation='vertical', nb_labels=3,label_fmt='%.2f')
#     cb.label_text_property.color =(0,0,0)
    

    mlab.draw()
    

    img1=mlab.screenshot(figure=maya,mode='rgba',antialiased=True)
    mlab.view(azimuth=0, elevation=90)
    mlab.figure(bgcolor=(0, 0, 0))
    ### clear figure
    mayavi.mlab.clf()
    
    f = mlab.gcf()
    cam = f.scene.camera
    cam.zoom(1.1)
    mlab.draw()
    img2=mlab.screenshot(figure=maya,mode='rgba',antialiased=True)
    ### clear figure
    mayavi.mlab.clf()
    return img1,img2

In [ ]:
### save KL images plotted on surfaces from oh_mayavi function to pngs 
def save_plots(a,b,name):
    plt.subplot(1,2,1)
    plt.imshow(b)
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(a)
    plt.axis('off')
    plt.subplots_adjust(hspace = -0.2)
    plt.savefig(f'leave_one_out_pngs/{name}.png',bbox_inches='tight',dpi=800,facecolor='w')
    plt.clf()



#### comment in for right hemisphere 
# def save_plots(a,b,name):
#     plt.subplot(2,2,1)
#     plt.imshow(a)
#     plt.axis('off')
#     plt.subplot(1,2,2)
#     plt.imshow(b)
#     plt.axis('off')
#     plt.subplots_adjust(hspace = -0.2)
    
#     plt.savefig(f'leave_one_out_pngs/{name}.png',bbox_inches='tight',dpi=800)
#     plt.clf()

# save_plots(p_all1,p_all2,'R_passoc')

Set up is now complete. Now we'll calculate the KL divergence between the pig and human using connectivity blueprints

In [ ]:
print('Calculate the KL divergence between the pig and human using the global connectivity BP with all 27 tracts') 
KL=calc_kl(pig_27,hum_27)
h_all=KL.min(axis=0)
p_all=KL.min(axis=1)
# KL=np.divide()
hmax=KL.min(axis=0).max()
pmax=KL.min(axis=1).max()
print(f'The range of the minimum KL using all tracts divergence for the pig is between 0 and {np.max(p_all):.2f}')
print(f'The range of the minimum KL divergence using all tracts for the human is between 0 and {np.max(h_all):.2f}')

In [ ]:
print('Calculating the KL divergence between the pig and human using the connectivity BP with no projection tracts')
hum_proj=remove_tracts(hum_27,['atr_l','atr_r', 'cst_l', 'cst_r','or_l', 'or_r', 'ptr_l', 'ptr_r', 'str_l', 'str_r'])
pig_proj=remove_tracts(pig_27,['atr_l','atr_r', 'cst_l', 'cst_r','or_l', 'or_r', 'ptr_l', 'ptr_r', 'str_l', 'str_r'])
KL_proj=calc_kl(pig_proj,hum_proj)
p_proj=KL_proj.min(axis=1)
h_proj=KL_proj.min(axis=0)
print(f'The range of the minimum KL using all tracts divergence for the pig is between 0 and {np.max(p_proj):.2f}')
print(f'The range of the minimum KL divergence using all tracts for the human is between 0 and {np.max(h_proj):.2f}')

In [ ]:
print('Calculating the KL divergence between the pig and human using the connectivity BP with no commisural tracts')
hum_comm=remove_tracts(hum_27,[ 'fma', 'fmi','mcp'])
pig_comm=remove_tracts(pig_27,['fma', 'fmi','mcp' ])
KL_comm=calc_kl(pig_comm,hum_comm)
p_comm=KL_comm.min(axis=1)
h_comm=KL_comm.min(axis=0)
print(np.max(p_comm))
print(np.max(h_comm))

In [ ]:
print('Calculate the KL divergence between the pig and human using the connectivity BP with no association tracts')
hum_assoc=remove_tracts(hum_27,['ifo_l', 'ifo_r', 'ilf_l', 'ilf_r','unc_l', 'unc_r' ])
pig_assoc=remove_tracts(pig_27,['ifo_l', 'ifo_r', 'ilf_l', 'ilf_r','unc_l', 'unc_r' ])
KL_assoc=calc_kl(pig_assoc,hum_assoc)
p_assoc=KL_assoc.min(axis=1)
h_assoc=KL_assoc.min(axis=0)
print(f'The range of the minimum KL divergence with the association tracts removed for the pig is between 0 and {np.max(p_assoc):.2f}')
print(f'The range of the minimum KL divergence with the association tracts removedfor the human is between 0 and {np.max(h_assoc):.2f}')



In [ ]:
print('Calculating the KL divergence between the pig and human using the connectivity BP with no limbic tracts')
hum_limbic=remove_tracts(hum_27,['cbd_l', 'cbd_r', 'cbt_l', 'cbt_r', 'cbp_l','fx_l', 'fx_r' ])
pig_limbic=remove_tracts(pig_27,['cbd_l', 'cbd_r', 'cbt_l', 'cbt_r', 'cbp_l','fx_l', 'fx_r' ])
KL_limbic=calc_kl(pig_limbic,hum_limbic)
p_limbic=KL_limbic.min(axis=1)
h_limbic=KL_limbic.min(axis=0)
print(f'The range of the minimum KL divergence with the limbic tracts removed for the pig is between 0 and {np.max(p_limbic):.2f}')
print(f'The range of the minimum KL divergence with the limbic tracts removedfor the human is between 0 and {np.max(h_limbic):.2f}')



Having calculated the KL divergence matrices between the pgi and human using the global blueprint and leave-one-out groups, we'll now determine the spatial correlation between different sets of blueprints to determine the role each tract group plays in driving the overall KL divergence between the pig and human

In [ ]:
##### plot the scatter plots and calculate r between the global and leave one out groups 
def plot_r_bps(A,B,out):
    A = A[~np.isnan(A)]
    B = B[~np.isnan(B)]
    m,b,r,p,st=scipy.stats.linregress(A,B)
    plt.figure(figsize=(4,2))
    ax=sn.regplot(A,B,scatter_kws={'s':3,'alpha':0.5,'color':'lightsteelblue'},
                  line_kws={'color':'mediumseagreen'})
    ax.text(0.05, 0.9,f'r={r:.2f}', transform=ax.transAxes, fontsize=14,
            verticalalignment='top')
    plt.savefig(f'leave_one_out_pngs/{out}.pdf',bbox_inches='tight',dpi=800,facecolor='w')
    return m,b,r

In [ ]:
plot_r_bps(h_all,h_proj,'h_proj+h_all')
plot_r_bps(h_all,h_comm,'h_comm+h_all')
plot_r_bps(h_all,h_assoc,'h_assoc+h_all')
plot_r_bps(h_all,h_limbic,'h_limbic+h_all')

plot_r_bps(p_all,p_proj,'p_proj+p_all')
plot_r_bps(p_all,p_comm,'p_comm+p_all')
plot_r_bps(p_all,p_assoc,'p_assoc+p_all')
plot_r_bps(p_all,p_limbic,'p_limbic+p_all')

The code used to generate the surface figures in the manuscript are generated by running the oh_mayavi function. 
Each run of oh_mayavi takes in a surface, and a single vector representing the data to be plotted on it.
The output of each run are two figure objects which are saved as images. <br /><bt />
The following cell plots the minimum KL divergence of the global connectivity blueprints containing all tracts, and the leave-one-out blueprints where a single tract group was left out. <br /> 
As usual this is for the left hemisphere only. To run on the right, comment out the left and comment in the right

In [ ]:
import vtk
# from mayavi import mlab



In [ ]:
# ### plot the lateral and axial views of the surfaces in mayavi 
# #### plotting human

##### note that it is best to run each set of tracts one by one commenting in and out for right now. 
from mayavi import mlab

h_proj2,h_proj1=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/L.rhum.inflated.surf.gii',h_proj,'terrain',0,hmax,'./surfaces/labels/L.hum.subcort.label')

h_comm2,h_comm1=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/L.rhum.inflated.surf.gii',h_comm,'terrain',0,hmax,'./surfaces/labels/L.hum.subcort.label')

h_assoc2,h_assoc1=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/L.rhum.inflated.surf.gii',h_assoc,'terrain',0,hmax,'./surfaces/labels/L.hum.subcort.label')

h_limbic2,h_limbic1=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/L.rhum.inflated.surf.gii',h_limbic,'terrain',0,hmax,'./surfaces/labels/L.hum.subcort.label')

h_all2,h_all1=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/L.rhum.inflated.surf.gii',h_all,'terrain',0,hmax,'./surfaces/labels/L.hum.subcort.label')



# ###### plotting pig
p_proj2,p_proj1=oh_mayavi('./surfaces/pig_surfaces/lh.inflated10k.surf.gii',p_proj,'terrain',0,pmax,'./surfaces/labels/L.pig.subcort.label')
p_comm2,p_comm1=oh_mayavi('./surfaces/pig_surfaces/lh.inflated10k.surf.gii',p_comm,'terrain',0,pmax,'./surfaces/labels/L.pig.subcort.label')
p_assoc2,p_assoc1=oh_mayavi('./surfaces/pig_surfaces/lh.inflated10k.surf.gii',p_assoc,'terrain',0,pmax,'./surfaces/labels/L.pig.subcort.label')
p_limbic2,p_limbic1=oh_mayavi('./surfaces/pig_surfaces/lh.inflated10k.surf.gii',p_limbic,'terrain',0,pmax,'./surfaces/labels/L.pig.subcort.label')
p_all2,p_all1=oh_mayavi('./surfaces/pig_surfaces/lh.inflated10k.surf.gii',p_all,'terrain',0,pmax,'./surfaces/labels/L.pig.subcort.label')

##### save the mayavi outputs to pngs 


save_plots(h_proj1,h_proj2,'L_h_proj')
save_plots(h_comm1,h_comm2,'L_hcomm')
save_plots(h_assoc1,h_assoc2,'L_hassoc')
save_plots(h_limbic1,h_limbic2,'L_hlimbic')
save_plots(h_all1,h_all2,'L_hall')



# ## comment in to run on right 

##### plot the lateral and axial views of the surfaces in mayavi 
##### plotting human
# h_proj1,h_proj2=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/R.rhum.inflated.surf.gii',h_proj,'terrain',0,hmax,'./surfaces/labels/R.hum.subcort.label')
# h_comm1,h_comm2=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/R.rhum.inflated.surf.gii',h_comm,'terrain',0,hmax,'./surfaces/labels/R.hum.subcort.label')
# h_assoc1,h_assoc2=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/R.rhum.inflated.surf.gii',h_assoc,'terrain',0,hmax,'./surfaces/labels/R.hum.subcort.label')
# h_all1,h_all2=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/R.rhum.inflated.surf.gii',h_all,'terrain',0,hmax,'./surfaces/labels/R.hum.subcort.label')
# ###### plotting pig
# p_proj1,p_proj2=oh_mayavi('./surfaces/pig_surfaces/rh.inflated10k.surf.gii',p_proj,'terrain',0,hmax,'./surfaces/labels/R.pig.subcort.label')
# p_comm1,p_comm2=oh_mayavi('./surfaces/pig_surfaces/rh.inflated10k.surf.gii',p_comm,'terrain',0,hmax,'./surfaces/labels/R.pig.subcort.label')
# p_assoc1,p_assoc2=oh_mayavi('./surfaces/pig_surfaces/rh.inflated10k.surf.gii',p_assoc,'terrain',0,hmax,'./surfaces/labels/R.pig.subcort.label')
# p_assoc1,p_assoc2=oh_mayavi('./surfaces/pig_surfaces/rh.inflated10k.surf.gii',p_limbic,'terrain',0,hmax,'./surfaces/labels/R.pig.subcort.label')
# p_all1,p_all2=oh_mayavi('./surfaces/pig_surfaces/rh.inflated10k.surf.gii',p_all,'terrain',0,hmax,'./surfaces/labels/R.pig.subcort.label')

###### comment in for right hemisphere (run one by one)
# save_plots(p_proj2,p_proj1,'R_p_proj')
# save_plots(p_comm2,p_comm1,'R_pcomm')
# save_plots(p_assoc2,p_assoc1,'R_passoc')
# save_plots(p_limbic1,p_limbic2,'R_plimbic')
# save_plots(p_all2,p_all1,'R_pall')

# save_plots(h_proj2,h_proj1,'R_h_proj')
# save_plots(h_comm2,h_comm1,'R_hcomm')
# save_plots(h_assoc2,h_assoc1,'R_hassoc')
# save_plots(h_limbic1,h_limbic2,'R_hlimbic')
# save_plots(h_all2,h_all1,'R_hall')






So far we have generated the minimum KL divergences between the global connectivity blueprint and the leave one out groups, and calculated their spatial correlation to one another. Now we calculate the change in KL divergence or what we refer to as ∆KL in the manuscript

In [ ]:
difHallHproj=h_all-h_proj
difHallHcomm=h_all-h_comm
difHallHassoc=h_all-h_assoc
difHallHlimbic=h_all-h_limbic

difPallPproj=p_all-p_proj
difPallPcomm=p_all-p_comm
difPallPassoc=p_all-p_assoc
difPallPlimbic=p_all-p_limbic

In [ ]:
print('changes in the minimum KL by tract group -- human ')
print(f'∆KL projection tract max is {np.nanmax(difHallHproj)}')
print(f'∆KL comm tract max is {np.nanmax(difHallHcomm)}')
print(f'∆KL asociaiton tract max is {np.nanmax(difHallHassoc)}')
print(f'∆KL limbic tract max is {np.nanmax(difHallHlimbic)}')
Hmax_dif=np.nanmax(difHallHlimbic)

In [ ]:
print('changes in the minimum KL by tract group -- human ')
print(f'∆KL projection tract max is {np.nanmax(difPallPproj)}')
print(f'∆KL comm tract max is {np.nanmax(difPallPcomm)}')
print(f'∆KL asociaiton tract max is {np.nanmax(difPallPassoc)}')
print(f'∆KL limbic tract max is {np.nanmax(difPallPlimbic)}')
Pmax_dif=np.nanmax(difPallPlimbic)

In [ ]:
###### save and plot in the left 

#### do the no projection 
p_dif1,p_dif2=oh_mayavi('./surfaces/pig_surfaces/lh.inflated10k.surf.gii'\
                        ,difPallPproj,'CMRmap',0,Pmax_dif,\
                        './surfaces/labels/L.pig.subcort.label')
save_plots(p_dif2,p_dif1,'L_p_diff_proj')

h_dif1,h_dif2=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/L.rhum.inflated.surf.gii',\
                           difHallHproj,'CMRmap',0,Hmax_dif,'./surfaces/labels/L.hum.subcort.label')
save_plots(h_dif2,h_dif1,'L_h_diff_proj')

##### do the no commisural 
p_dif1,p_dif2=oh_mayavi('./surfaces/pig_surfaces/lh.inflated10k.surf.gii'\
                        ,difPallPcomm,'CMRmap',0,Pmax_dif,\
                        './surfaces/labels/L.pig.subcort.label')
save_plots(p_dif2,p_dif1,'L_p_diff_comm')

h_dif1,h_dif2=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/L.rhum.inflated.surf.gii',\
                           difHallHcomm,'CMRmap',0,Hmax_dif,'./surfaces/labels/L.hum.subcort.label')
save_plots(h_dif2,h_dif1,'L_h_diff_comm')

#### do the no association 
p_dif1,p_dif2=oh_mayavi('./surfaces/pig_surfaces/lh.inflated10k.surf.gii'\
                        ,difPallPassoc,'CMRmap',0,Pmax_dif,\
                        './surfaces/labels/L.pig.subcort.label')
save_plots(p_dif2,p_dif1,'L_p_diff_assoc')

h_dif1,h_dif2=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/L.rhum.inflated.surf.gii',\
                           difHallHassoc,'CMRmap',0,Hmax_dif,'./surfaces/labels/L.hum.subcort.label')
save_plots(h_dif2,h_dif1,'L_h_diff_assoc')

##### do the no limbic 
p_dif1,p_dif2=oh_mayavi('./surfaces/pig_surfaces/lh.inflated10k.surf.gii'\
                        ,difPallPlimbic,'CMRmap',0,Pmax_dif,\
                        './surfaces/labels/L.pig.subcort.label')
save_plots(p_dif2,p_dif1,'L_p_diff_limbic')

h_dif1,h_dif2=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/L.rhum.inflated.surf.gii',\
                           difHallHlimbic,'CMRmap',0,Hmax_dif,'./surfaces/labels/L.hum.subcort.label')
save_plots(h_dif2,h_dif1,'L_h_diff_limbic')

########### Comment in to do the right 

# #### do the no projection 
# p_dif1,p_dif2=oh_mayavi('./surfaces/pig_surfaces/rh.inflated10k.surf.gii'\
#                         ,difPallPproj,'CMRmap',0,Pmax_dif,\
#                         './surfaces/labels/R.pig.subcort.label')
# save_plots(p_dif2,p_dif1,'R_p_diff_proj')

# h_dif1,h_dif2=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/R.rhum.inflated.surf.gii',\
#                            difHalrhproj,'CMRmap',0,Hmax_dif,'./surfaces/labels/R.hum.subcort.label')
# save_plots(h_dif2,h_dif1,'R_h_diff_proj')

##### do the no commisural 
# p_dif1,p_dif2=oh_mayavi('./surfaces/pig_surfaces/rh.inflated10k.surf.gii'\
#                         ,difPallPcomm,'CMRmap',0,Pmax_dif,\
#                         './surfaces/labels/R.pig.subcort.label')
# save_plots(p_dif2,p_dif1,'R_p_diff_comm')

# h_dif1,h_dif2=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/R.rhum.inflated.surf.gii',\
#                            difHalrhcomm,'CMRmap',0,Hmax_dif,'./surfaces/labels/R.hum.subcort.label')
# save_plots(h_dif2,h_dif1,'R_h_diff_comm')

# #### do the no association 
# p_dif1,p_dif2=oh_mayavi('./surfaces/pig_surfaces/rh.inflated10k.surf.gii'\
#                         ,difPallPassoc,'CMRmap',0,Pmax_dif,\
#                         './surfaces/labels/R.pig.subcort.label')
# save_plots(p_dif2,p_dif1,'R_p_diff_assoc')

# h_dif1,h_dif2=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/R.rhum.inflated.surf.gii',\
#                            difHalrhassoc,'CMRmap',0,Hmax_dif,'./surfaces/labels/R.hum.subcort.label')
# save_plots(h_dif2,h_dif1,'R_h_diff_assoc')

# ##### do the no limbic 
# p_dif1,p_dif2=oh_mayavi('./surfaces/pig_surfaces/rh.inflated10k.surf.gii'\
#                         ,difPallPlimbic,'CMRmap',0,Pmax_dif,\
#                         './surfaces/labels/R.pig.subcort.label')
# save_plots(p_dif2,p_dif1,'R_p_diff_limbic')

# h_dif1,h_dif2=oh_mayavi('./surfaces/rmars-comparing-connectivity-blueprints-surfaces/R.rhum.inflated.surf.gii',\
#                            difHalrhlimbic,'CMRmap',0,Hmax_dif,'./surfaces/labels/R.hum.subcort.label')
# save_plots(h_dif2,h_dif1,'R_h_diff_limbic')


In [ ]:
##### set colorbars scales by gorup max difference 
fig, ax = plt.subplots(figsize=(10,1))
fig.subplots_adjust(bottom=0.5)

cmap = mpl.cm.CMRmap
norm = mpl.colors.Normalize(vmin=0, vmax=Pmax_dif)
cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap,
                                norm=norm,
                                orientation='horizontal')
# cb1.outline.set_edgecolor('k')
ax.tick_params(axis='y', colors='white')


# ax.remove()
# cb1.set_label('Human KL Divergence')
plt.plot()
plt.savefig('leave_one_out_pngs/L_pigDif_colorbar.png',
            bbox_inches='tight',facecolor='w',edgecolor='w')


fig, ax = plt.subplots(figsize=(10,1))
fig.subplots_adjust(bottom=0.5)

cmap = mpl.cm.CMRmap
norm = mpl.colors.Normalize(vmin=0, vmax=Hmax_dif)
cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap,
                                norm=norm,
                                orientation='horizontal')
# cb1.outline.set_edgecolor('k')
ax.tick_params(axis='y', colors='white')


# ax.remove()
# cb1.set_label('Human KL Divergence')
plt.plot()
plt.savefig('leave_one_out_pngs/L_humDif_colorbar.png',
            bbox_inches='tight',facecolor='w',edgecolor='w')

The figures for parts A and B for figures 6-9 are generated through the above cells. <br />
Now all that's left is part C which contains the cortical tract projections. <br / >
Part C was plotted using workbench_view as part of the HCP workbench software package,and the tract projections can be loaded using the included spec file. <br /> 
However, we here show an example of how a connectivity blueprint consists of the tract projections to the surface as shown in part C of figured 6-9

In [ ]:
### an example plotting the anterior thalamic radiation
plotting.view_surf('surfaces/rmars-comparing-connectivity-blueprints-surfaces/L.rhum.midthickness.surf.gii',hum_27['atr_l'],cmap='videen_style')

In [ ]:
#### doing the same in the pig. 
plotting.view_surf('surfaces/pig_surfaces/lh.graymid10k.surf.gii',pig_27['atr_l'],cmap='videen_style')

In [ ]:
####The columns of the Blueprint were saved as giftis with the following function and visualized in workbench for the manuscript
def save_gifti(data,out):
    gi = nib.gifti.GiftiImage()
    da = nib.gifti.GiftiDataArray(np.float32(data), intent=0)
    gi.add_gifti_data_array(da)
    nib.save(gi,f'{out}.func.gii')

In [ ]:
#### an example 
save_gifti(pig_27['atr_l'],'tract_projections/lh_atr_pig_nb_example')